# DECSKS-14: Two species Vlasov simulations

## A. Motivation

The aim of this notebook is to develop a top-level orchestrator routine that employs DECSKS-2.0 (runs through the evolution of one species) to evolve two distribution functions. That is, the objective is to build a framework that allows the data from each species evolution to be accessed by the other in an obvious way. The boundary conditions are not the focus in this notebook and we will take them to be periodic as that has already been proofed and set up. The conductor routine will take a form similar to

    # DECSKS2S.main (2S = "two species")

    import DECSKS2S 
    
    # the densities will be fi[t,x,vx], and fe[t,x,vx]
    # initialize both distributions f_i[0, x, vx], f_e[0,x,vx]
    # instantiate a common set of phase space variables (x,vx)
    
    for n in t.stepnumbers:
        
        fe, fi = DECSKS2S.lib.split.scheme(f_e, f_i, *args)
        
        # generate plots and compute diagnostics
        
    # END
    
Other changes forseen (rough/naive form shown below):

    ## changes in lib.split.scheme
    
        # if vx advection
    
        n = lib.fieldsolvers.single_integration(fi, fe, of = x, wrt = vx)
        E = lib.fieldsolvers.Gauss(n, *args) 


# B. Normalizing electron and ion Vlasov equations

We cast the Vlasov equations for both species in terms of set of common normalized variables. The normalizations are chosen according to characteristic scales of the plasma species which requires the most resolution so that the increments of time, length, and velocity in the numerical simulation suit the dynamics without having to partition the mesh quantities into unreasonably small fractions (that beyond inconvenience, can easily challenge machine precision). Here, the relevant scales are fixed by the characteristic electric field length, i.e. the Debye length, peculiar velocities will be traded for velocities relative to an average (thermal) velocity of the more mobile species (electrons), and the time measurements will be measured as multiples of the plasma period (time to traverse one circuit of the inertial (Debye) length).


## B.1.A For an evolution equation for any species $\alpha$

In standard units, we have for a species $\alpha$ the 1D1V Vlasov equation:

$$\partial_t f_{\alpha} + v\partial_x f_{\alpha} + \frac{q_{\alpha}E}{m_{\alpha}} \partial_v f_{\alpha} = 0$$

We elect to normalize to natural units, i.e. multiples of characteristic parameters of a plasma species. Define normalized (tilde) quantities according to:

\begin{eqnarray*}
t = \frac{1}{\omega_{p\alpha}}\tilde{t} & \longrightarrow & \partial_t = \omega_{p\alpha}\partial_{\tilde{t}} \qquad \omega_{p\alpha} = \sqrt{\frac{n_{\alpha} q_{\alpha}^2}{\epsilon_0m_{\alpha}}} \\
x = \lambda_D \tilde{x} & \longrightarrow & \partial_x = \frac{1}{\lambda_D}\partial_{\tilde{x}} \qquad \lambda_D = \sqrt{\frac{\epsilon_0 kT_{\alpha}}{n_{\alpha}q_{\alpha}}} \\
v = v_{T\alpha} \tilde{v} & \longrightarrow & \partial_v = \frac{1}{v_{T\alpha}}\partial_{\tilde{v}} \qquad v_{T\alpha} = \sqrt{\frac{kT_{\alpha}}{m_{\alpha}}}\\
\end{eqnarray*}

Subsequent normalizations may be needed, and will be decided if it is seen to be appropriate in the following. Substituting all of the above into the species Vlasov equation:

\begin{eqnarray*}
\partial_t f_{\alpha} + v\partial_x f_{\alpha} + \frac{q_{\alpha}E}{m_{\alpha}} \partial_v f_{\alpha} & = & 0 \\
\left(\omega_{\alpha}\partial_{\tilde{t}}\right) f_{\alpha} + \left(\tilde{v}v_{T\alpha}\right) \left(\frac{1}{\lambda_D}\partial_{\tilde{x}}\right)f_{\alpha} + \frac{q_{\alpha}E}{m_{\alpha}}\left(\frac{1}{v_{T\alpha}}\partial_{\tilde{v}}\right) f_{\alpha} = 0 \\
\underbrace{\lambda_D\omega_{\alpha}}_{=\, v_{T\alpha}}\partial_{\tilde{t}} f_{\alpha} + \tilde{v}v_{T\alpha}\partial_{\tilde{x}}f_{\alpha} + \frac{q_{\alpha}E}{m_{\alpha}}\frac{\lambda_D}{v_{T\alpha}}\partial_{\tilde{v}} f_{\alpha} = 0 \\
v_{T\alpha}\partial_{\tilde{t}} f_{\alpha} + \tilde{v}v_{T\alpha}\partial_{\tilde{x}}f_{\alpha} + \frac{q_{\alpha}E}{m_{\alpha}}\frac{\lambda_D}{v_{T\alpha}}\partial_{\tilde{v}} f_{\alpha} = 0 \\
\partial_{\tilde{t}} f_{\alpha} + \tilde{v}\partial_{\tilde{x}}f_{\alpha} + \left(\frac{q_{\alpha}\lambda_D}{m_{\alpha}v_{T\alpha}^2}\right) E \partial_{\tilde{v}} f_{\alpha} = 0 \\
\end{eqnarray*}

Hence, as anticipated we do require one more normalized quantity. It is natural to normalize the electric field here as:

$$\tilde{E} = \left(\frac{|q_{\alpha}|\lambda_D}{m_{\alpha}v_{T\alpha}^2}\right) E = \frac{E}{\bar{E}}$$

where $\bar{E} = m_{\alpha}v_{T\alpha}^2 / (|q_{\alpha}|\lambda_D) = \mathrm{[force / charge]}$ has the same units as the electric field. Note, the silent decision to normalize by the magnitude is obvious but has the effect that the charge remaining in the equation is of unit charge, i.e. the above version of the Vlasov equation then is rewritten in terms of this as:


$$\partial_{\tilde{t}} f_{\alpha} + \tilde{v}\partial_{\tilde{x}}f_{\alpha} + \frac{q_{\alpha}}{|q_{\alpha}|}\underbrace{\left(\frac{|q_{\alpha}|\lambda_D}{m_{\alpha}v_{T\alpha}^2}\right) E}_{=\, \bar{E}E \equiv \tilde{E}} \partial_{\tilde{v}} f_{\alpha} = 0$$

$$\partial_{\tilde{t}} f_{\alpha} + \tilde{v}\partial_{\tilde{x}}f_{\alpha} + \frac{q_{\alpha}}{|q_{\alpha}|}\tilde{E} \partial_{\tilde{v}} f_{\alpha} = 0 $$


or, relabelling $\tilde{w} \rightarrow w$ for $w = \{t,x,v,E\}$, we understand the equation

$$\partial_t f_{\alpha} + v\partial_xf_{\alpha} +  \frac{q_{\alpha}}{|q_{\alpha}|}E \partial_v f_{\alpha} = 0 $$

measures time in multiples of plasma periods ($\sim \omega_{p\alpha}^{-1}$) for the charged species $\alpha$, $x$ is a measure of distance as multiples of Debye lengths $\lambda_D$ for species $\alpha$, velocity is measured as multiples of the species thermal velocity $v_{T\alpha}$, and the electric field is in multiples of $m_{\alpha}v_{T\alpha}^2 / (\lambda_D q_{\alpha})$, which can be interpreted as the force field per unit charge of species $\alpha$ averaged over one Debye length at its average speed.

## B.1.B Normalization of the electron and (singly charged) ion Vlasov equations

The equations evolve alongside each other. We require a reference equation that indicates the measurement of common $t, x, v, E$ variables, whereafter we can insist on the same measurements for the second equation that ultimatley will have small modifications given intertial differences ($m_{\alpha}$ and $q_{\alpha}$).

We need to choose to measure on time scales and lengths according to the shortest physical scales between $\alpha = e,i$, thus it is the electron evolution that decides our normalization. Accordin\gly, from the above we permit $\alpha = e$, noting $q_e / |q_e| = -1$ and write down the normalized electron Vlasov equation:

$$\boxed{\partial_t f_{e} + v\partial_xf_{e} - E \partial_v f_{e} = 0} \qquad \underline{\text{Normalized electron Vlasov equation}} $$

To find the normalization required for the second equation, we take a few steps backwards and start wtih the original equation for clarity. The ion equation (single charged) takes the form:

$$\partial_t f_{i} + v\partial_x f_{i} + \frac{q_iE}{m_i} \partial_v f_i = 0$$

we use consistent normalizations as with the electron equation:

\begin{eqnarray*}
t = \frac{1}{\omega_{pe}}\tilde{t} & \longrightarrow & \partial_t = \omega_{pe}\partial_{\tilde{t}} \qquad \omega_{pe} = \sqrt{\frac{n_{e} q_{e}^2}{\epsilon_0m_{e}}} \\
x = \lambda_D \tilde{x} & \longrightarrow & \partial_x = \frac{1}{\lambda_D}\partial_{\tilde{x}} \qquad \lambda_D = \sqrt{\frac{\epsilon_0 kT_{e}}{n_{e}q_{e}}} \\
v = v_{Te} \tilde{v} & \longrightarrow & \partial_v = \frac{1}{v_{Te}}\partial_{\tilde{v}} \qquad v_{Te} = \sqrt{\frac{kT_{e}}{m_{e}}}\\
\end{eqnarray*}

\begin{eqnarray*}
\partial_t f_{i} + v\partial_x f_{i} + \frac{q_{i}E}{m_{i}} \partial_v f_{i} & = & 0 \\
\left(\omega_{pe}\partial_{\tilde{t}}\right) f_{i} + \left(\tilde{v}v_{Te}\right) \left(\frac{1}{\lambda_D}\partial_{\tilde{x}}\right)f_{i} + \frac{q_{i}E}{m_{i}}\left(\frac{1}{v_{Te}}\partial_{\tilde{v}}\right) f_i = 0 \\
\underbrace{\lambda_D\omega_{pe}}_{=\, v_{Te}}\partial_{\tilde{t}} f_i + \tilde{v}v_{Te}\partial_{\tilde{x}}f_i + \frac{q_{i}E}{m_{i}}\frac{\lambda_D}{v_{T\alpha}}\partial_{\tilde{v}} f_i = 0 \\
v_{Te}\partial_{\tilde{t}} f_i + \tilde{v}v_{Te}\partial_{\tilde{x}}f_i + \frac{q_{i}E}{m_{i}}\frac{\lambda_D}{v_{Te}}\partial_{\tilde{v}} f_i = 0 \\
\partial_{\tilde{t}} f_i + \tilde{v}\partial_{\tilde{x}}f_i + \left(\frac{q_{i}\lambda_D}{m_{i}v_{Te}^2}\right) E \partial_{\tilde{v}} f_i = 0 \\
\partial_{\tilde{t}} f_i + \tilde{v}\partial_{\tilde{x}}f_i + \frac{q_i}{|q_i|}\left(\frac{|q_{i}|\lambda_D}{m_{i}v_{Te}^2}\right) E \partial_{\tilde{v}} f_i = 0 
\end{eqnarray*}

Recall our normalization for the electric field are characterized not by ion quantities, but by the electron inertia:

$$\tilde{E} = \left(\frac{|q_{e}|\lambda_D}{m_{e}v_{Te}^2}\right) E = \frac{E}{\bar{E}}$$

We replace $E$ with $E = \bar{E}\tilde{E} = \left(\frac{m_{e}v_{Te}^2}{|q_{e}|\lambda_D}\right) \tilde{E}$ noting that $|q_e| = |q_i|$ for our singly charged case so that above, the electric field term becomes:

$$ \frac{q_i}{|q_i|}\left(\frac{|q_{i}|\lambda_D}{m_{i}v_{Te}^2}\right) E =  \frac{q_i}{|q_i|}\left(\frac{|q_{i}|\lambda_D}{m_{i}v_{Te}^2}\right) \left(\frac{m_{e}v_{Te}^2}{|q_{e}|\lambda_D}\right)\tilde{E} = \underbrace{\frac{q_i}{|q_i|}}_{=\, 1}\underbrace{\left(\frac{|q_i|}{|q_e|}\right)}_{=\, 1}\underbrace{\left(\frac{\lambda_D}{\lambda_D}\right)}_{=\, 1}\underbrace{\left(\frac{v_{Te}^2}{v_{Te}^2}\right)}_{=\, 1}\left(\frac{m_e}{m_i}\right)\tilde{E} = \frac{m_e}{m_e}\tilde{E} = \frac{1}{\mu}\tilde{E}$$

Where $\mu = m_i / m_e \simeq 1836$ has been defined. Finaly, relabelling the normalized variables as $\tilde{E} \rightarrow E$, $\tilde{t} \rightarrow t$, $\tilde{x} \rightarrow x$, $\tilde{v} \rightarrow v$ as before with the understanding that it is a normalized quantity, the ion species equation is found to be:

$$\boxed{\partial_t f_{i} + v\partial_x f_{i} + \frac{1}{\mu}E\partial_v f_i = 0} \qquad \underline{\text{Normalized (singly charged) ion Vlasov equation}}$$. 

Thus, as needed, it is the acceleration terms that are distinct among the species:

$$a_{x,e} = -E, \qquad \underline{\text{electron acceleration}}$$

$$a_{x,i} = \frac{1}{\mu}E, \qquad \underline{\text{ion acceleration}}$$

so that the ion species is less mobile by a factor of $1 / \mu \simeq 1 / 1836$. And, we can write a unified $n$-species set of Vlasov equations for some species $\beta$ as

$$\partial_t f_{\beta} + v_x\partial_x f_{\beta} + a_{x,\beta}\partial_v f_{\beta} = 0$$

where the normalizations are in terms of the most mobile species $\alpha$, and

$$a_{x,\beta} = \begin{cases}
\text{sign}(q_{\alpha})Z_{\alpha}E & \text{if } \beta = \alpha \\
& \\
\text{sign}(q_{\beta})\frac{Z_{\beta}}{\mu_{\beta\alpha}}E, \qquad \mu_{\beta\alpha} = m_{\beta} / m_{\alpha} & \text{else}
\end{cases}
$$

# B.2 Normalizing distribution functions (e.g. initial conditions) to be consistent with the set of normalized variables $(\tilde{t}, \tilde{x}, 
\tilde{v}_x)$

The associated normalizations from using the above need to be same variables that appear in distribution functions we may decide as initial or boundary conditions. As above, when evolving two distribution functions, it is useful to designate the species which requires resolving these parameters most significantly as a reference, and to see the effects that has on the distribution for the other species' function. Suppose the normalizations have been chosen according to a species $\alpha$ (i.e. is the controller), then distribution functions for another species $\beta$ (or same $\beta = \alpha$), needs to be recasted consistently:

$$f_{\beta}(t,x,v) \mapsto f_{\alpha}(\omega_{p\alpha}^{-1}t, \lambda_D x, v_{T\alpha} v)$$

Take an example of an unnormalized Maxwellian centered at $v = 0$ (we save normalization for after the final form is obtained)

$$f_{\beta}(t,x,v) = \exp \left(-\frac{m_{\beta}v^2}{2T_{\beta}}\right)$$

Putting this in terms of the normalized variables we have $v \mapsto v_{T\alpha} v = \sqrt{T_{\alpha} / m_{\alpha}} v$ so that the measurement is that velocity is multiples of the $\alpha$ thermal velocity.

$$f_{\beta}(t,x,v) = \exp \left(-\frac{m_{\beta}v_{T\alpha}^2 v^2}{2T_{\beta}}\right) =\exp \left(-\frac{1}{2}\frac{m_{\beta}}{m_{\alpha}}\frac{T_{\alpha}}{T_{\beta}}v^2\right) = \exp \left(-\frac{1}{2}\frac{m_{\beta}}{m_{\alpha}}\frac{T_{\alpha}}{T_{\beta}}v^2\right)$$

Define the mass ratio $\mu = m_{\beta} / m_{\alpha}$ (i.e. just as before we define the factor to be such that the species that sets the physical scales, $\alpha$ is in the denominator, and define the temperature ratio $\tau = T_{\beta} / T_{\alpha}$, the above is equivalent to:

$$f_{\beta}(t,x,v) = \exp \left(-\frac{1}{2}\frac{\mu}{\tau}v^2\right)$$

For $\beta = \alpha$, we have the unnormalized function

$$f_{\alpha}(t,x,v) = \exp \left(-\frac{1}{2}v^2\right)$$

We can normalize both functions over all possible velocity values. For the $\alpha$ distribution, we prepend a factor of $1 / \sqrt{2\pi}$

$$\boxed{f_{\alpha}(t,x,v) = \frac{1}{\sqrt{2\pi}}\exp \left(-\frac{1}{2}v^2\right)}$$

For the $\beta$ distribution we prepend a factor of $\sqrt{\mu / \tau}$ on top of the $1 / \sqrt{2\pi}$ term:

$$\boxed{f_{\beta}(t,x,v) = \frac{1}{\sqrt{2\pi \tau / \mu}} \exp \left(-\frac{v^2}{\sqrt{2\tau / \mu}}\right)}$$

Note, to shift the velocity distribution, we do so by the normalized coordinate "$v$" $\equiv \tilde{v}$ (unlabelled in the boxed equations for brevity), where $\tilde{v} = v_{peculiar} / v_{T\alpha}$. In the above, shifting the distribution by $n\in\mathbb{Z}$ corresponds to shifting by a multiple of the thermal velocity $v_{T\alpha}$ in both of the boxed equations.

A special case of interest because of what appears in the Bohm criterion is shifting by the sound speed:

$$c_s \simeq \sqrt{\frac{T_{\alpha}}{m_{\beta}}} = \sqrt{\frac{T_{\alpha}}{m_{\alpha}}\frac{m_{\alpha}}{m_{\beta}}} = \frac{v_{T\alpha}}{\sqrt{\mu}}$$

So the normalized quantity we add is some $\tilde{c}_s$,

$$\tilde{c}_s = \frac{c_s}{v_{T\alpha}} = \frac{1}{\sqrt{\mu}}$$

So, shifting any distribution $\beta$ by some multiple $n\in\mathbb{R}$ of the plasma sound velocity is done by:

$$f_{\beta}(t,x,v) = \frac{1}{\sqrt{2\pi \tau / \mu}} \exp \left(-\frac{(v - n\tilde{c}_s)^2}{2\tau / \mu}\right), \qquad \tilde{c}_s = \frac{1}{\sqrt{\mu}}, n\in\mathbb{R}\qquad \underline{\text{Shifted by a multiple of the sound speed in normalized variables}}$$

In general, we can of course always write

$$f_{\beta}(t,x,v) = \frac{1}{\sqrt{2\pi \tau / \mu}} \exp \left(-\frac{(v - v_0)^2}{\sqrt{2\tau / \mu}}\right), \qquad v_0 =v_{peculiar} / v_{T\alpha} \quad \underline{\text{in normalized coordinates for arbitrary } v_0\in\mathbb{R}}$$

In the below setup, we do not current interest in one distribution over another. Hence, we will use centered Maxwellians just to show the evolution of $f_i$ and $f_e$ is possible (goal of this noteboook). We also choose this in order to have symmetric conditions so that a periodic plasma can be considered as this is the simplest situation to analyze and DECSKS-2.0 with periodic boundatry conditions has been benchmarked sufficiently. It is not a question of comparing Fourier vs. finite difference corrections in this notebook, as we have those methods benchmarked (arbitrarily, we will elect to use Fourier based corrections). The only goal is to simulate the self-consistent interplay between ions and electrons.

## B.3 Consequences of have two different grid resolutions for ions and electrons

In any simulation, it is necessary to have the same configuration domain extents: $x\in \mathcal{D}_x$; however we are free to choose the number of gridpoints needed. Since electronic length scales are shorter than the characteristic ionic length scales, we may for example choose to have $(N_x)_{f_i} > (N_x)_{f_e}$ since the ions are significantly colder and move much smaller distances as compared to electrons in the same time step. While it is clear the configuration domain need to be equal so that physically we have electrons and ions occupying the same space, it is clear that they in general populate different regions of the overall phase sapce becuase of the large differences in velocity values taken between them. Thus, we do not require the same velocity domains. In fact, we should restrict the domain if we can in order to not waste storage and simulation expense. Thus, we may choose for example the electron velocity grid as $v_e\in [-V_e, V_e]$, whereas the ion velocities may lie within $v_i\in [-V_i, V_i]$ (neither need be symmetric, this is just an example) wehre $V_i \neq V_e$. In general, there is also no consequence in having the grid points being different as well. For example, a coarser ion velocity mesh might be desired compared to the electron velocity grid ($(N_v)_{f_e} > (N_v)_{f_i}$ if ionzation physics is included. This is because of the important role of resolving the ionzation contribution from the high energy tail of electrons as compared to the slower ions (the largest source of ionzation is from electron-neutral collisions), hence we require adequately resolving the high energy tail with more grid points. For example, an isotropic production rate $K_{iz}$ over unit volume $[\mathrm{L^3 / s}]$ is represented as [2]:

$$K_{iz} = \frac{\int_{4\pi} d\Omega \int_{v_{iz}}^{\infty} dv v^2 (v\sigma_{iz}(v) f_e(v))}{\int d^3 x \int d^3 v f_e(x,v)} \sim \frac{1}{n_e} \int_{v_{ix}}^{\infty} dv \, v^3 f_e(v) \sigma_{iz}(v)$$

Where only sufficiently energetic electrons can cause impact-ionization, $v_{iz} = \sqrt{2e\mathcal{E}_{iz} / m_e}$ in standard units for some ionization energy $\mathcal{E}_{iz}$ for the particular neutral species. Hence, it is clear that ionization depends on the distribution function skewness, so the resolution of the high energy tail is motivated sufficiently. It is easy to overestimate or underestimate the number of ions borne from impact collisions if the mesh is not sufficiently resolved, it depends on the shape of the cross-section.

A sticking point that remains when deciding to have different grids is resolving how to compute a moment quantity such as

$$n = \int dv \, [f_i(t,x,v) - f_e(t,x,v)] =  \int_{v_i\in\mathcal{D}_{vi}} dv_i \, f_i(t,x_i,v_i) - \int_{v_e\in\mathcal{D}_{ve}} dv_e\, f_e(t,x_e,v_e)]$$

The actual integrations over velocity spare are no problem, i.e. 
$$\phantom{whitespace}$$

$n_i(t,x_i) = \int_{v_i\in\mathcal{D}_{vi}} dv_i \, f_i(t,x_i,v_i)$ = <code> numpy.sum(fi[n,:,:], axis = 1)*vx_i.width</code>
$n_e(t,x_e)  = \int_{v_e\in\mathcal{D}_{ve}} dv_i \, f_i(t,x_e,v_e)$ = <code> numpy.sum(fe[n,:,:], axis = 1)*vx_e.width</code>

However, if the resulting number densities ni and ne do not have the same number $N_x$ gridpoints, it is not straightforward to compute the required difference to obtain the charge density:

        n = ni - ne    # alignment error if ni.shape != ne.shape
        
The clear way to compute this difference is to construct:

$$\text{dim} (n) = \text{max}(\text{dim}(n_i), \text{dim}(n_e))$$

suppose $\text{dim}(n_i) < \text{dim}(n_e)$, we can compute at each n[k] (we omit the time argument, it is understood this is at a certain time $t= t^n$) according to

$$n(x_e) = n_i(x_e) - n_e(\tilde{x}_i) , \qquad \qquad \text{for } \tilde{x}_i \colon \{\min_{\forall x_i}|\{x_i\} - x_e|\}_{x_e\in\mathcal{D}_{xe}}, \quad \text{and } x_e\in\mathcal{D}_{xe} $$

This can be used to find $E$ and thus $a_{x,e}$ for each abscissa value $x_e\in\mathcal{D}_{xe}$. Then, the reverse can be done to find the density at every $x_i\in\mathcal{D}_{xi}$ which is used to calculate $E$ and thus $a_{x,i}$ at every abscissa $x_i\in\mathcal{D}_{xi}$. Depending on the coarseness of one grid relative to another, this could tarnish previous efforts to make the scheme overall higher order since it is a low order nearest gridpoint interpolation. Further, the cost of doing this is discouraging foir the little benefit. Thus, this naive route seems to already to be unsalvageable. 

A better way would be to fit curves to the densities $\mathcal{P}[n_e]$ and $\mathcal{P}[n_i]$ sample the gridpoints to compute

$$n(x_i) = \mathcal{P}[n_i](x_i) - \mathcal{P}[n_e](x_i), \qquad \qquad \text{for all } x_i\in\mathcal{D}_{xi}$$

which is appropriately sized and evaluated at the correct points to calculate $E(x_i) \Rightarrow a_{x,i}(x_i)$. Next, sample

$$n(x_e) = \mathcal{P}[n_i](x_e) - \mathcal{P}[n_e](x_e), \qquad \qquad \text{for all } x_e\in\mathcal{D}_{xe}$$

which is also appropriately defined to compute $E(x_e) \Rightarrow a_{x,e}(x_e)$. Thus, the stepthrough can continue easily.

<b>We delay this consideration for the time being, perhaps in a later notebook we will revisit this idea. At this juncture, we suffice to choose $(N_x)_{f_i} = (N_x)_{f_e}$ and $(N_v)_{f_i} = (N_v)_{f_e}$</b>


For equal gridpoints, integrating to obtain important moments is done as usual: 

    n = numpy.sum(fi[n,:,:] - fe[n,:,:], axis = 1)*vx.width
    
Note this would work even with different number of velocity grid points, the only requirement here that axis = 0 has the same number (corresponds to the configuration x). In the case of different Nv, the phase space variable vx would have two varieites: vx_i and vx_e for each species with the relevant attributes stored (required two different input files, this is no problem, but also something that needs to be juggled).

    n = numpy.sum(fi[n,:,:], axis = 1)*vx_i.width - numpy.sum(fe[n,:,:], axis = 1)*vx_e.width

Notwithstanding, we use the former version and plan on having Nx_e = Nx_i, and Nvx_i = Nvx_e.

# C. Model problem

Thus, we examine the evolution of the following set of Vlasov equations for an $L$-periodic plasma:

$$\partial_t f_e + v_x\partial_x f_e + \partial_x\phi \partial_{v_x} f_e = 0, \qquad\qquad x\in [-L, L], v_x\in [-V, V], t\in [0,T], \qquad L,T,V\in\mathbb{R}$$

$$\partial_t f_i + v_x\partial_x f_i - \frac{1}{\mu}\partial_x\phi \partial_{v_x} f_i = 0 \qquad \qquad x\in [-L, L], v_x\in [-V, V], t\in [0,T], \qquad x_w,T\in\mathbb{R}, \quad \mu = m_i / m_e$$

Where $L = 20$, $V$ is to be determined (different for both species), $T$ will be determined.

alternatively, we can replace the potential with the electric field as $\partial_x \phi = -E$. In this notebook we look at periodic boundary conditions:

<u>Boundary conditions</u> (Periodic)

<ol>
<li> Density: $f_{e,i}(t,x + 2L, v_x) = f_e(t, x, v_x)$, $f_{e,i}(t,x,v_x + 2V) = f_{e,i}(t,x,v_x)$
<li>Potential: $\phi (t,x + 2L) = \phi (t, x)$
</ol>

<u>Initial conditions</u>

<ol>
<li> $f_e(0,x, v_x) = f_{e}(t,x,v) = \frac{1}{\sqrt{2\pi}} \exp \left(-\frac{1}{2}v^2\right)$
$${}$$
<li> $f_i(0,x, v_x) = f_{i}(t,x,v) = \frac{1}{\sqrt{2\pi \tau / \mu}} \exp \left(-\frac{v^2}{2\tau / \mu}\right), \qquad \tau = 1/30, \mu = 1836.15267389 \text{ (hydrogen) }$
</ol>


# Other rough changes to be finalized to code

    ## new method: lib.boundaryconditions.density.absorbing
    
        # f[:, v > 0] = 0, this is an efficient call in numpy incidentally
        
    ## new method: lib.boundaryconditions.potential.chargedplate
    
        sigma = 0
        for v in vx.prepoints:
            sigma += np.sum(vx[v]*f_i[0,v] - vx[v]*f_e[0,v], axis = 1)*vx.width # i = 0 at wall
            
        E[at wall] = 1 / 2. * sigma
        

# Staggering integrators $\Phi_h$

The evolution of each distribution cannot happen simultaneously. We stagger the evolution of each naturally by time substep.

We advect positions according to their prepoint velocity mesh values. Whenever we have a vx advection, we push the velocity prepoints according to the prepoint accelerations, where the accelerations are proportional to the electric field. We require the most current locations of all charged particles so that the electric field can be kept up to date and evolve the velocity cells accurately. Hence, the staggered time integrators take the form

In general, we have the following pseudocode stepthrough:

    FOR n in range(timesteps):
    
        if advecting x
    
        advect both fe and fi over time substep according to mesh values vx
    
        elif advecting vx
    
            calculate charge density n
            calculate field E
            assign accelerations ax_e, ax_i
        
            advect both fe and fi over time substep according to ax_e and ax_i mesh values
        

<u>Example</u>: in LF2 splitting we have (1,1/2, 1)$\Delta t$ as the time steps, where we interleave x-vx-x advection

    # LF2 pseudocode
    
    (1) x advection over 1/2*dt
    
        advect fe(t + 1/2 dt,x,vx) = fe(t, x - vx*1/2*dt, vx)
        advect fi(t + 1/2 dt. x. vx) = fi(t, x - vx*1/2*dt, vx)
        
    (2) vx advection over 1*dt
    
        n = single_integration(fi - fe, of = x, wrt = vx)
        E = DECSKS2S.lib.fieldsolvers.Gauss1D1V(n, *args)
        ax_e = -E
        ax_i = 1/mu * E
        
        advect fe(t + 1*dt, x, vx) = fe(t, x, v - ax_e*1*dt)
        advect fi(t + 1*dt, x, vx) = fi(t, x, v - ax_i*1*dt)
        
    (3) x advection over 1/2*dt
    
        t' = t + 1/2*dt
        advect fe(t' + 1/2 dt,x,vx) = fe(t', x - v*1/2*dt, vx)
        advect fi(t '+ 1/2 dt. x. v)x = fi(t', x - v*1/2*dt, vx)      

In previous work (cf. <a href = "https://github.com/dsirajud/Preliminary-examination">Prelim document and slides</a>), we had labellled the split integrators as $\mathcal{X}^{\tau}$ and $\mathcal{V}^{\tau}$, for time step $\tau$. We see the above amounts to

<b>LF2 split</b> over time $t \mapsto t + \tau$:

(1) $x(t) \mapsto x + v(t) \tau / 2$

$$f_e(x(t + \tau / 2), v) = \mathcal{X}^{\tau /2} [ f_e(t, x, v)]$$
$$f_i(x(t + \tau / 2), v) = \mathcal{X}^{\tau /2} [ f_i(t, x, v)]$$
 
(2) $v(t) \mapsto v(t) + a_{x,(e,i)}\tau$

where we firsit calculate $E(t^n, x(t + \tau / 2))$, then assign $a_{x,e} = -E$, $a_{x,i} = (1/ \mu )E$, and advect according to:

$$f_e(x(t + \tau / 2) , v(t + \tau )) = \mathcal{V}^{\tau} [ f_e(t, x(t + \tau / 2), v)]$$
$$f_i(x(t + \tau / 2), v(t + \tau)) = \mathcal{V}^{\tau} [ f_i(t, x(t + \tau / 2), v)]$$


(3)  $x + v(t)\tau / 2 \mapsto (x(t) + v(t)\tau/2 ) + v(t + \tau )\tau / 2$

$$f_e(t + \tau , x, v) \equiv f_e(x(t + \tau), v(t + \tau)) = \mathcal{X}^{\tau /2} [ f_e(t, x(t + \tau / 2), v(t + \tau ))]$$
$$f_i(t + \tau , x, v) \equiv f_i(x(t + \tau), v(t + \tau)) = \mathcal{X}^{\tau /2} [ f_i(t, x(t + \tau / 2), v(t + \tau ) )]$$

To still make this compact further, define $\mathcal{E}$ as the electric field functional, where we write $\mathcal{E}[f_i - f_e]$. Then, we are able to succinctly frame any split scheme:

### Any split scheme

Call the exact integrator $\Phi^{T}$ over a simulation time $T$. Then, we numerically approximate:

$$\boxed{\Phi^T \simeq \Phi^T_h  = \Pi_{n = 1}^{N_t}\Pi_{s = 1}^S \mathcal{V}_i^{d_s\tau_n}(\mathcal{E}_s)[f_i] \circ\mathcal{V}_e^{d_s\tau_n}(\mathcal{E}_s)[f_e]\circ\mathcal{X}^{c_s\tau_n}[f_i]\circ\mathcal{X}^{c_s\tau_n}[f_e]} \qquad \underline{\text{General two-species split scheme}}$$

Where

$$\sum_{n=1}^{N_t} \tau_n = T, \sum_{s = 1}^{S} c_s = \sum_{s = 1}^S d_s = 1$$

And, the Lie operators can be defined in terms of their action:

\begin{eqnarray*}
f_{e,i}(t + c_s\tau, x, v = \text{const}) & = &\mathcal{X}^{c_s\tau}[f_{e,i}(t,x,v)] = f_{e,i}(t, x - vc_s\tau, v) \\
f_{i}(t + d_s\tau, x = \text{const}, v)
) & = &\mathcal{V}_i^{d_s\tau}(\mathcal{E}_s)f_{i}(t,x,v) = f_{i}(t, x, v + (1 / \mu )\mathcal{E}[f_i - f_e]d_s\tau ) \\
f_{e}(t + d_s\tau, x = \text{const}, v) & = & \mathcal{V}_e^{d_s\tau}(\mathcal{E}_s)f_{e}(t,x,v) = f_{e}(t, x, v - \mathcal{E}[f_i - f_e]d_s\tau )
\end{eqnarray*}

And,

$$\mathcal{E}_s[f_i - f_e] \equiv \mathcal{E}_s[f_i(x(t + c_s\tau), v) - f_e(x(t + c_s\tau), v)] = \partial_x^{-1} \int dv\, [f_i(x(t + c_s\tau),v) - f_e(x(t + c_s\tau),v)]$$

is the electric field as calculated after a time substep $c_s\tau$ of the total time time step $\tau$. In other words, it is the electric field as evaluated from the $x$-positions after the previous $\mathcal{X}$-advection. The operator $\partial_x^{-1}$ is the suitable inversion operator for Gauss' law.

Recall that foundationally, these operators take the form  $\mathcal{X}^{c_s\tau} \equiv e^{\Lambda_xc_s\tau}$  and  $\mathcal{V}^{d_s\tau} \equiv e^{\Lambda_v d_s \tau}$, for the split bracket operators $\Lambda_x \equiv \left\{\frac{m_{e,i}v^2}{2}, \cdot\right\}$, and $\Lambda_v \equiv \left\{\pm\phi, \cdot\right\}$ follow from the partition of the explicit Hamiltonian $H$ for the electrostatic system. Different split schemes are obtained by seeking $S$-fold interleavings of these two operators so that the original (noncommuting) operator $e^{\{H,\cdot\}\tau}$ splits the time $\tau$ into $S$ stages so that the desired number of order conditions are satisfied when unfolding the exponentiated Lie operator $e^{\{H, \cdot\}\tau}$ per the Baker-Campbell-Hausdorff formula.




We can extend this easily to $m$-species rather than just two:

$$\boxed{\Phi^T \simeq \Phi^T_h  = \Pi_{n = 1}^{N_t}\Pi_{s = 1}^S \mathcal{V}_{\alpha 1}^{d_s\tau_n}(\mathcal{E}_s)[f_{\alpha 1}] \circ\mathcal{V}_{\alpha 2}^{d_s\tau_n}(\mathcal{E}_s)[f_{\alpha 2}]\circ \\
\ldots\circ\mathcal{V}_{\alpha m}^{d_s\tau_n}(\mathcal{E}_s)[f_{\alpha m}]\circ\mathcal{X}_{\alpha 1}^{c_s\tau_n}[f_{\alpha 1}]\circ\mathcal{X}^{c_s\tau_n}[f_{\alpha 2}]\circ\ldots\circ\mathcal{X}^{c_s\tau_n}[f_{\alpha m}]} \qquad \underline{\text{General $m$-species split scheme}}$$

# References

[1] Lieberman